# Autores

**Helena Ferrero Chaves**

**María Pecker Gayarre**

# Motivación

El valor real de los datos recolectados en bruto solo puede ser comprendido una vez transformados en información.

Así, para garantizar la eficiencia logística de los sistemas de transporte y dar un mejor servicio al usuario es necesario recolectar, tratar y procesar una gran cantidad de datos, con el objetivo de transformar estos datos en información relevante.
Y en dicha transformación, la estadística es de gran importancia para el buen 
funcionamiento del sistema de transporte de una ciudad.

Después, esa información puede ser utilizada para mejorar la eficiencia de los procesos, la oferta al cliente y, en general, para tomar mejores decisiones de negocio.


# Objetivos

El objetivo general de este trabajo es demostrar el potencial que tienen las nuevas tecnologías, el "Big Data" y el análisis de datos para dar respuesta a los problemas que surgen dentro del ámbito de la movilidad urbana y mejorar la eficiencia de los sistemas de transporte dentro de las ciudades.

Se pretende abordar este objetivo mediante la elaboración de un caso práctico con datos reales del sistema BiciMAD.

# Dataset

**BiciMAD. Disponibilidad de unidades (bicicletas) del servicio público de bicicleta eléctrica**

*Web*: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=7547ff52e4a4f410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

**Periodo de los datos**: entre el 10/08/2015 y el 30/10/2022

**Estructura datos**: 
https://datos.madrid.es/FWProjects/egob/Catalogo/Transporte/Bici/Ficheros/Bicimad_Estructura_FICHERO_DATOS.pdf

# Planteamiento

En base a los datos que se manejan, se plantean las siguientes estadísticas agrupadas por categorías.

#### Uso de abonado anual y ocasional por año

*   **Promedio de uso del servicio con abono anual por año.**
*   **Promedio de uso del servicio con abono ocasional por año.**

#### Uso de bicicletas y disponibilidad por año

*   **Promedio de uso de las bicicletas por año.** *Porcentaje de tiempo que las bicicletas están en uso.*
*   **Promedio de disponibilidad de las bicicletas por año.** *Porcentaje de tiempo que las bicicletas están disponibles.*

#### Bicicletas a pleno rendimiento y bicicletas a plena disponibilidad por año

*   **Bicicletas a pleno rendimiento por año.** *Aquí, en vez de dar un número de horas de uso de bicicletas por año, se ha preferido convertir ese dato en la cantidad de bicicletas que se han usado las 24 horas de los 365 días del año. Evidentementen, no quiere decir que se use una bicicleta durante un año seguido.*
*   **Bicicletas a plena disponibilidad por año.** *Aquí, en vez de dar un número de horas de uso de bicicletas por año, se ha preferido convertir ese dato en la cantidad de bicicletas que han estado disponibles las 24 horas de los 365 días del año.*

#### Bicicletas a pleno rendimiento por año, con abono anual y con abono ocasional

*   **Bicicletas a pleno rendimiento por año con abono anual.** *Aquí, en vez de dar un número de horas de uso de bicicletas por año, se ha preferido convertir ese dato en la cantidad de bicicletas que se han usado con abono anual las 24 horas de los 365 días del año.*
*   **Bicicletas a pleno rendimiento por año con abono ocasional.** *Aquí, en vez de dar un número de horas de uso de bicicletas por año, se ha preferido convertir ese dato en la cantidad de bicicletas que se han usado con abono ocasional las 24 horas de los 365 días del año.*

# Implementación

## Preparar el entorno de ejecución en Colaboraty

Instalar la librería findspark

In [1]:
!pip install -q findspark

Instalar pyspark

In [2]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Crear sesión de Spark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Crear sparkContext

In [4]:
sc = spark.sparkContext

Montar cuenta de Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Estadísticas

### Crear dataframe con los datos

Crear un Dataframe a partir del fichero 'bici_disponibilidad 10.08.2015_30.10.2022.csv'

In [11]:
df_dispo = spark.read.option('header','true').option('delimiter', ';').option('inferSchema','true').csv('./drive/MyDrive/Colaboratory/Proyecto Bicimad/data/bici_disponibilidad 10.08.2015_30.10.2022.csv')
df_dispo.printSchema()

root
 |-- DIA: string (nullable = true)
 |-- HORAS_TOTALES_USOS_BICICLETAS: double (nullable = true)
 |-- HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN_ANCLAJES: double (nullable = true)
 |-- TOTAL_HORAS_SERVICIO_BICICLETAS: double (nullable = true)
 |-- MEDIA_BICICLETAS_DISPONIBLES: double (nullable = true)
 |-- USOS_ABONADO_ANUAL: integer (nullable = true)
 |-- USOS_ABONADO_OCASIONAL: integer (nullable = true)
 |-- TOTAL_USOS: integer (nullable = true)



Añadir al Dataframe la columna AÑO. Sus datos se obtienen de la columna DIA, que contiene una fecha

In [12]:
from pyspark.sql.functions import substring, col
df_dispo2 = df_dispo.withColumn('AÑO', substring(col('DIA'), 7, 4))
df_dispo2.printSchema()

root
 |-- DIA: string (nullable = true)
 |-- HORAS_TOTALES_USOS_BICICLETAS: double (nullable = true)
 |-- HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN_ANCLAJES: double (nullable = true)
 |-- TOTAL_HORAS_SERVICIO_BICICLETAS: double (nullable = true)
 |-- MEDIA_BICICLETAS_DISPONIBLES: double (nullable = true)
 |-- USOS_ABONADO_ANUAL: integer (nullable = true)
 |-- USOS_ABONADO_OCASIONAL: integer (nullable = true)
 |-- TOTAL_USOS: integer (nullable = true)
 |-- AÑO: string (nullable = true)



### Uso de abonado anual y ocasional por año

Obtener promedio

In [13]:
from pyspark.sql.functions import round, sum

df = df_dispo2.groupBy('AÑO').agg(
    round(sum('USOS_ABONADO_ANUAL') / sum('TOTAL_USOS') * 100, 2).alias('Promedio USOS_ABONADO_ANUAL'),
    round(sum('USOS_ABONADO_OCASIONAL') / sum('TOTAL_USOS') * 100, 2).alias('Promedio USOS_ABONADO_OCASIONAL')
    ).sort('AÑO')
df.show() 

+----+---------------------------+-------------------------------+
| AÑO|Promedio USOS_ABONADO_ANUAL|Promedio USOS_ABONADO_OCASIONAL|
+----+---------------------------+-------------------------------+
|2015|                      97.68|                           2.32|
|2016|                      97.89|                           2.11|
|2017|                      97.75|                           2.25|
|2018|                      98.16|                           1.84|
|2019|                      98.78|                           1.22|
|2020|                      99.33|                           0.67|
|2021|                      99.54|                           0.46|
|2022|                      99.68|                           0.32|
+----+---------------------------+-------------------------------+



Mostrar tabla

In [14]:
import pyspark.pandas as ps

# Convertir pyspark.sql.dataframe.DataFrame to pyspark.pandas.frame.DataFrame
df_tabla = ps.DataFrame(df).set_index('AÑO')
df_tabla

,Promedio USOS_ABONADO_ANUAL,Promedio USOS_ABONADO_OCASIONAL
AÑO,,
2015,97.68,2.32
2016,97.89,2.11
2017,97.75,2.25
2018,98.16,1.84
2019,98.78,1.22
2020,99.33,0.67
2021,99.54,0.46
2022,99.68,0.32


Mostrar gráficas

In [15]:
df1 = ps.DataFrame(df)
df1.plot.line(x='AÑO', y='Promedio USOS_ABONADO_ANUAL')

In [16]:
df2 = ps.DataFrame(df)
df2.plot.line(x='AÑO', y='Promedio USOS_ABONADO_OCASIONAL') 

In [17]:
from plotly.subplots import make_subplots

df3 = ps.DataFrame(df)
fig = (make_subplots(rows=2, cols=1, subplot_titles=("Promedio USOS_ABONADO_ANUAL por año", "Promedio USOS_ABONADO_OCASIONAL por año"))
       .add_trace(df3.plot.bar(x='AÑO', y='Promedio USOS_ABONADO_ANUAL').data[0], row=1, col=1)
       .add_trace(df3.plot.bar(x='AÑO', y='Promedio USOS_ABONADO_OCASIONAL').data[0], row=2, col=1))
fig.update_xaxes(title_text="Año", row=1, col=1)
fig.update_xaxes(title_text="Año", row=2, col=1)
fig.update_yaxes(title_text="Promedio", row=1, col=1)
fig.update_yaxes(title_text="Promedio", row=2, col=1)
fig  

#### **Análisis de resultados**

Se observa que el uso de abonos ocasionales se ha ido decrementando con los años. Mientras que el uso de los abonos anuales se ha ido incrementando.

### Uso de bicicletas y disponibilidad por año

Obtener promedio

In [23]:
from pyspark.sql.functions import round, sum

df = df_dispo2.groupBy('AÑO').agg(
    round(sum('HORAS_TOTALES_USOS_BICICLETAS') / sum('TOTAL_HORAS_SERVICIO_BICICLETAS') * 100, 2).alias('Promedio USOS_BICICLETAS'),
    round(sum('HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN_ANCLAJES') / sum('TOTAL_HORAS_SERVICIO_BICICLETAS') * 100, 2).alias('Promedio DISPONIBILIDAD_BICICLETAS')
    ).sort('AÑO')
df.show() 

+----+------------------------+----------------------------------+
| AÑO|Promedio USOS_BICICLETAS|Promedio DISPONIBILIDAD_BICICLETAS|
+----+------------------------+----------------------------------+
|2015|                   12.79|                             87.21|
|2016|                    6.05|                             93.95|
|2017|                    6.07|                             93.93|
|2018|                    6.05|                             93.95|
|2019|                    6.34|                             93.66|
|2020|                    6.29|                             93.71|
|2021|                    5.53|                             94.47|
|2022|                    5.26|                             94.74|
+----+------------------------+----------------------------------+



Mostrar tabla

In [24]:
import pyspark.pandas as ps

# Convertir pyspark.sql.dataframe.DataFrame to pyspark.pandas.frame.DataFrame
df_tabla = ps.DataFrame(df).set_index('AÑO')
df_tabla

,Promedio USOS_BICICLETAS,Promedio DISPONIBILIDAD_BICICLETAS
AÑO,,
2015,12.79,87.21
2016,6.05,93.95
2017,6.07,93.93
2018,6.05,93.95
2019,6.34,93.66
2020,6.29,93.71
2021,5.53,94.47
2022,5.26,94.74


Mostrar gráficas

In [25]:
df1 = ps.DataFrame(df)
df1.plot.line(x='AÑO', y='Promedio USOS_BICICLETAS')

In [26]:
df2 = ps.DataFrame(df)
df2.plot.line(x='AÑO', y='Promedio DISPONIBILIDAD_BICICLETAS') 

In [27]:
from plotly.subplots import make_subplots

df3 = ps.DataFrame(df)
fig = (make_subplots(rows=2, cols=1, subplot_titles=("Promedio USOS_BICICLETAS por año", "Promedio DISPONIBILIDAD_BICICLETAS por año"))
       .add_trace(df3.plot.bar(x='AÑO', y='Promedio USOS_BICICLETAS').data[0], row=1, col=1)
       .add_trace(df3.plot.bar(x='AÑO', y='Promedio DISPONIBILIDAD_BICICLETAS').data[0], row=2, col=1))
fig.update_xaxes(title_text="Año", row=1, col=1)
fig.update_xaxes(title_text="Año", row=2, col=1)
fig.update_yaxes(title_text="Promedio", row=1, col=1)
fig.update_yaxes(title_text="Promedio", row=2, col=1)
fig  

#### **Análisis de resultados**

Se observa que el uso de bicicletas se ha mantenido a lo largo de los años, salvo en los dos últimos años del estudio que ha descendido levemente. Mientras que la disponibilidad de bicicletas se ha incrementando en la misma proporción en los dos últimos años.

### Bicicletas a pleno rendimiento y bicicletas a plena disponibilidad por año

Obtener estadística

In [ ]:
from pyspark.sql.functions import round, sum

df = df_dispo2.groupBy('AÑO').agg(
    round(sum('HORAS_TOTALES_USOS_BICICLETAS') / 24 / 365,2).alias('Bicicletas usadas el año completo'),
    round(sum('HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN_ANCLAJES') / 24 / 365, 2).alias('Bicicletas sin usar el año completo')
    ).sort('AÑO')
df.show() 

+----+---------------------------------+-----------------------------------+
| AÑO|Bicicletas usadas el año completo|Bicicletas sin usar el año completo|
+----+---------------------------------+-----------------------------------+
|2015|                            78.28|                             533.99|
|2016|                           112.73|                             1752.0|
|2017|                           110.08|                            1702.08|
|2018|                           116.36|                            1807.11|
|2019|                           129.63|                            1915.87|
|2020|                           133.23|                            1984.75|
|2021|                           141.81|                            2420.72|
|2022|                           126.72|                            2282.57|
+----+---------------------------------+-----------------------------------+



Mostrar tabla

In [ ]:
import pyspark.pandas as ps

# Convertir pyspark.sql.dataframe.DataFrame to pyspark.pandas.frame.DataFrame
df_tabla = ps.DataFrame(df).set_index('AÑO')
df_tabla

,Bicicletas usadas el año completo,Bicicletas sin usar el año completo
AÑO,,
2015,78.28,533.99
2016,112.73,1752.00
2017,110.08,1702.08
2018,116.36,1807.11
2019,129.63,1915.87
2020,133.23,1984.75
2021,141.81,2420.72
2022,126.72,2282.57


Mostrar gráficas

In [ ]:
df1 = ps.DataFrame(df)
df1.plot.line(x='AÑO', y='Bicicletas usadas el año completo')

In [ ]:
df2 = ps.DataFrame(df)
df2.plot.line(x='AÑO', y='Bicicletas sin usar el año completo') 

#### **Análisis de resultados**

Se observa que el tiempo de uso de las bicicletas ha ido incrementándose anualmente. Salvo en el último, año que ha descendido un poco. Y lo mismo le ha ocurrido a la disponibilidad de bicicletas, que ha ido incrementándose anualmente de manera leve, y en el último año ha descendido un poco.

### Bicicletas a pleno rendimiento por año, con abono anual y con abono ocasional

Obtener estadística

In [ ]:
from pyspark.sql.functions import round, sum

df = df_dispo2.groupBy('AÑO').agg(
    round(sum('HORAS_TOTALES_USOS_BICICLETAS') * round(sum('USOS_ABONADO_ANUAL') / sum('TOTAL_USOS') * 100, 2) / 100 / 24 / 365, 2).alias('Bicicletas usadas el año completo con abono anual'),
    round(sum('HORAS_TOTALES_USOS_BICICLETAS') * round(sum('USOS_ABONADO_OCASIONAL') / sum('TOTAL_USOS') * 100, 2) / 100 / 24 / 365,2).alias('Bicicletas usadas el año completo con abono ocasional')
    ).sort('AÑO')
df.show() 

+----+-------------------------------------------------+-----------------------------------------------------+
| AÑO|Bicicletas usadas el año completo con abono anual|Bicicletas usadas el año completo con abono ocasional|
+----+-------------------------------------------------+-----------------------------------------------------+
|2015|                                            76.47|                                                 1.82|
|2016|                                           110.35|                                                 2.38|
|2017|                                            107.6|                                                 2.48|
|2018|                                           114.22|                                                 2.14|
|2019|                                           128.05|                                                 1.58|
|2020|                                           132.33|                                                 0.89|
|

Mostrar tabla

In [ ]:
import pyspark.pandas as ps

# Convertir pyspark.sql.dataframe.DataFrame to pyspark.pandas.frame.DataFrame
df_tabla = ps.DataFrame(df).set_index('AÑO')
df_tabla

,Bicicletas usadas el año completo con abono anual,Bicicletas usadas el año completo con abono ocasional
AÑO,,
2015,76.47,1.82
2016,110.35,2.38
2017,107.60,2.48
2018,114.22,2.14
2019,128.05,1.58
2020,132.33,0.89
2021,141.15,0.65
2022,126.31,0.41


Mostrar gráficas

In [ ]:
df1 = ps.DataFrame(df)
df1.plot.line(x='AÑO', y='Bicicletas usadas el año completo con abono anual')

In [ ]:
df2 = ps.DataFrame(df)
df2.plot.line(x='AÑO', y='Bicicletas usadas el año completo con abono ocasional') 

#### **Análisis de resultados**

Se observa que el tiempo de uso de las bicicletas con abono anual es muy superior al del abono ocasional. Y además, el primero se está incrementando con los años. Mientras que el segundo va decrementándose.

# Análisis de resultados

Van en la Implementación. Y dentro de ella, en las Estadísticas. Al final de cada estadística hay un apartado con nombre "Análisis de resultados".